In [ ]:
import pandas as pd
import urllib.parse
import gmaps
import gmaps.datasets
from IPython.display import display
import requests, json

In [ ]:
_2018_df = pd.read_csv("viol_data/viol_2018.csv")
_2019_df = pd.read_csv("viol_data/viol_2019.csv")
_2020_df = pd.read_csv("viol_data/viol_2020.csv")
_2021_df = pd.read_csv("viol_data/viol_2021.csv")
_2022_df = pd.read_csv("viol_data/viol_2022.csv")

In [ ]:
_frames = [_2018_df, _2019_df, _2020_df, _2021_df, _2022_df]
full_df = pd.concat(_frames)

In [ ]:
#drop any violations not issued by DoT (V)
full_df = full_df[full_df['Issuing Agency'] != 'P']

In [ ]:
#pd.set_option('display.max_columns', None)

In [ ]:
#columns to keep
_columns = ['Issue Date', 'Violation County', 'Street Name', 'Intersecting Street']

In [ ]:
#filter and clean data and columns
#
#
rlv_df = full_df[_columns].copy().reset_index(drop=True)

#replace NY, K with MN, BK
rlv_df.loc[rlv_df["Violation County"] == "NY", "Violation County"] = "MN"
rlv_df.loc[rlv_df["Violation County"] == "K", "Violation County"] = "BK"
rlv_df.loc[rlv_df["Violation County"] == "ST", "Violation County"] = "SI"
rlv_df['Violation County'] = rlv_df['Violation County'].fillna('SI')


#convert str date to datetime
rlv_df['Issue Date'] = pd.to_datetime(rlv_df['Issue Date'])

#get date elements
rlv_df['Day of Week'] = rlv_df['Issue Date'].dt.dayofweek
rlv_df['Month'] = rlv_df['Issue Date'].dt.month
rlv_df['Year'] = rlv_df['Issue Date'].dt.year

#clean street_name and intersecting_street data
rlv_df['Street Name'] = rlv_df['Street Name'].str.replace(r"\([A-Z]*\/*[A-Z]*\)*|\/", "")
rlv_df['Street Name'] = rlv_df['Street Name'].str.replace(r" +", " ")
rlv_df['Street Name'] = rlv_df['Street Name'].str.replace("@", "&")
rlv_df['Street Name'] = rlv_df['Street Name'].str.replace(")", "")

rlv_df['Intersecting Street'] = rlv_df['Intersecting Street'].str.replace(r"\([A-Z]\/*[A-Z]*\)*|\/[A-Z]|[A-Z]\/[A-Z]|\/", "")
rlv_df['Intersecting Street'] = rlv_df['Intersecting Street'].str.replace(r" +", " ")
rlv_df['Intersecting Street'] = rlv_df['Intersecting Street'].str.replace("@", "&")
rlv_df['Intersecting Street'] = rlv_df['Intersecting Street'].str.replace(")", "")

#join cross locations
rlv_df['Location'] = (rlv_df['Street Name'] + rlv_df['Intersecting Street'] + ", " + rlv_df['Violation County'])

In [ ]:
#confirm no rogue characters remain in the location strings
contains_paran = rlv_df[rlv_df['Location'].str.contains('\(|\)|\/', na=False)]
contains_paran.shape

In [ ]:
#get set of unique locations for geocoding
_loc_counts = rlv_df['Location'].value_counts().rename_axis('Location').reset_index(name='counts')

In [ ]:
#read in api key
_f_key = open("api_key.txt")
_api_key = _f_key.readline()

In [ ]:
# maps.googleapis.com/maps/api

# use google maps, geocode api to turn location_cross streets into lat/long points to plot



base_url = 'https://maps.googleapis.com/maps/api/geocode/json?address='

_coords = []
for idx in range(len(_loc_counts)):
    _query = _loc_counts.loc[idx, "Location"]
    _encoded_query = urllib.parse.quote(_query)
    _joined_url = base_url + _encoded_query + '&key=' + _api_key

    response = requests.get(_joined_url)
    _resp_data = response.json()
    if (len(_resp_data['results']) > 0):
        _lat = _resp_data['results'][0]['geometry']['location']['lat']
        _long = _resp_data['results'][0]['geometry']['location']['lng']
        _coords.append((_lat, _long))
    else:
        _coords.append((0, 0))

_loc_counts['coords'] = pd.Series(_coords)


# =================================================================================== #

#check for any missed locations
_loc_counts[_loc_counts['coords'] == (0, 0)]

#hey, only 3! pretty good api
#we can just look those up ourselves

# =================================================================================== #

_loc_err_1 = (40.74918603001331, -73.72914070015736)
_loc_err_2 = (40.7440379569156, -73.83718153784129)
_loc_err_3 = (40.78165396955172, -73.82575252278569)

_loc_counts.at[3, 'coords'] = _loc_err_1
_loc_counts.at[43, 'coords'] = _loc_err_2
_loc_counts.at[202, 'coords'] = _loc_err_3

#two places on Bronx River Parkway misplaced in Yonkers
_loc_counts.at[17, 'coords'] = (40.894697011169704, -73.86161051360368)
_loc_counts.at[91, 'coords'] = (40.894697011169704, -73.86161051360368)

In [ ]:
_loc_counts = pd.read_csv("locs_counts_coords.csv")

In [ ]:
#write _loc_counts to dictionary for matching in full dataframe
_loc_dict = _loc_counts.set_index('Location')['coords'].to_dict()

In [ ]:
#create coords column in dataframe
rlv_df['coords'] = ''

In [ ]:
#match locations in dataframe to dictionary, append coords
for idx in range(len(rlv_df)):
    _loc = rlv_df.loc[idx, "Location"]
    _coord = _loc_dict.get(_loc, 'Not Found')
    rlv_df.at[idx, "coords"] = _coord

In [ ]:
rlv_df.head()

In [ ]:
#_by_coords = _loc_counts.groupby(['coords'], as_index=False).agg({'counts': 'sum', 'Location': 'first'})
#_by_coords.sort_values(by='counts', ascending=False)

### Normalizing Violation Totals

In [ ]:
camera_counts = pd.read_csv("viol_data/camera_counts.csv")

In [ ]:
#boros: BK - BX - MN - QN - SI
_registered_drivers = pd.Series([1072118, 494367, 788888, 1199319, 312885])

In [ ]:
_by_boro = rlv_df.groupby("Violation County")["Year"].count().to_frame().rename(columns={'Year':'violations'})
#_by_boro['county'] = ['BK', 'BX', 'MN', 'QN', 'SI']
#_by_boro = _by_boro[['county', 'violations']]
#_by_boro.reset_index(drop=True)

In [ ]:
_temp = []
for idx, ele in enumerate(_by_boro['violations']):
    _temp.append(float(ele / _registered_drivers[idx]))
_by_boro['viols_nrml'] = _temp
_by_boro['density'] = camera_counts['density'].tolist()

In [ ]:
_by_boro

In [ ]:
501201 / _registered_drivers[0]

In [ ]:
# ============== split data by year ============== #
#
#
boros = ["BX", "BK", "MN", "QN", "SI"]

def split_by_boro(_boro, _df):
    _boro_df = pd.DataFrame(_df[_df['Violation County'] == _boro])
    return _boro_df

In [ ]:
_pre_covid_df = rlv_df[(rlv_df['Year'] == 2018) | (rlv_df['Year'] == 2019)]
_dur_covid_df = rlv_df[rlv_df['Year'] == 2020] 
_pst_covid_df = rlv_df[rlv_df['Year'] == 2021]

In [ ]:
_pst_covid_df.shape

In [ ]:
_yearly_dfs[0].head()

,Issue Date,Violation County,Street Name,Intersecting Street,Day of Week,Month,Year,Location,coords
0,2018-01-01,BK,4TH AVE & 60TH,ST,0,1,2018,"4TH AVE & 60THST, BK","(40.6408947, -74.01849729999999)"
1,2018-01-01,BK,WILLIAMSBURG ST,& FLUSHING AVE,0,1,2018,"WILLIAMSBURG ST & FLUSHING AVE, BK","(40.69800680000001, -73.9628506)"
2,2018-01-01,QN,ROCKAWAY BLVD,& BROOKVILLE BLVD,0,1,2018,"ROCKAWAY BLVD & BROOKVILLE BLVD, QN","(40.6397239, -73.74266260000002)"
3,2018-01-01,QN,KISSENA BLVD &,BOOTH MEMORIAL AVE,0,1,2018,"KISSENA BLVD &BOOTH MEMORIAL AVE, QN","(40.7429665, -73.8143118)"
4,2018-01-01,BK,HOWARD AVE & E,NEW YORK AVE,0,1,2018,"HOWARD AVE & ENEW YORK AVE, BK","(40.6682589, -73.9201265)"


In [ ]:
_2018_gb = _yearly_dfs[0].groupby(by='coords', as_index=False).agg({'Location':'first', 'Year':'sum'}).rename(columns={'Year':'total'})
_2018_gb

,coords,Location,total
0,"(40.5231708, -74.1885465)","LUTEN AVE & HYLAN BLVD, SI",84756
1,"(40.561147, -74.1698222)","ARTHUR KILL RD & RICHMOND AVE, SI",4788714
2,"(40.5637615, -74.1558671)","ARTHUR KILL RD & GIFFORDS LN, SI",3131936
3,"(40.5721723, -74.1081494)","HYLAN BLVD & BURBANK AVE, SI",843524
4,"(40.5747583, -73.9734752)","SURF AVE & W 5TH ST, BK",1775840
...,...,...,...
155,"(44.9377599, -93.2727598)","3RD AVE & 36THST, MN",3043144
156,"(44.9555309, -93.2740437)","2ND AVE & 26THST, MN",532752
157,"(44.977753, -93.2650108)","8TH AVE & 57THST, MN",12376394
158,"(46.729553, -94.6858998)","WEST ST & W HOUSTON ST, MN",9797390


In [ ]:
_2018_loc_counts = _yearly_dfs[0]['Location'].value_counts().rename_axis('Location').reset_index(name='counts')
_2019_loc_counts = _yearly_dfs[1]['Location'].value_counts().rename_axis('Location').reset_index(name='counts')
_2020_loc_counts = _yearly_dfs[2]['Location'].value_counts().rename_axis('Location').reset_index(name='counts')
_2021_loc_counts = _yearly_dfs[3]['Location'].value_counts().rename_axis('Location').reset_index(name='counts')

In [ ]:
_boro_yearly_dfs = []

for year_df in _yearly_dfs:
    for boro in boros:
        _boro_yearly_dfs.append(split_by_boro(boro, year_df))

In [ ]:
for idx, ele in enumerate(_boro_yearly_dfs):
    print(boros[idx % 5], ele.shape)
    if (idx % 5 == 4):
        print()

In [ ]:
_samples = []
for ele in enumerate(_boro_yearly_dfs):
    _samples.append(ele[1].sample(n=1000).reset_index(drop=True))

In [ ]:
_boro_yearly_dfs[0]['Location'].value_counts()

In [ ]:
#configure gmaps with api key
gmaps.configure(api_key = _api_key)

In [ ]:
new_york_coordinates = (40.67, -73.94)
fig = gmaps.figure(center=new_york_coordinates, zoom_level=10)

_pos = _by_coords['coords'].tolist()
_locs = _by_coords['Location'].tolist()
symbol_layer = gmaps.symbol_layer(_pos, hover_text=_locs)

fig.add_layer(symbol_layer)
fig

In [ ]:
#write out files
#

#write limited dataframe to csv
#rlv_df.to_csv('red_light_camera_tickets.csv')

#write location-coords dataframe (post query) to csv
_loc_counts.to_csv('locs_counts_coords.csv')